# Glacier edu module WIP
This is an introduction/walk through of the glacier module in the OGGM-Edu library.
At the moment it contains three classes: `GlacierBed`, `Glacier` and `GlacierCollection`. 

#### Interactive boxes PSA

- Green
<div class="alert alert-success">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Orange
<div class="alert alert-warning">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Red
<div class="alert alert-danger">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Blue
<div class="alert alert-info">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from oggm_edu import Glacier, GlacierCollection, GlacierBed, SurgingGlacier

## Glacier bed
The `GlacierBed` provides with a separate object for the bed of the glacier.
This is then passed to the initialisation of the glacier.
There are two ways to initialise a glacier bed:
- Pass single scalars to the arguments top, bottom and width.
    This creates a square bed.
- Pass multiple values (list/tuple) to altitudes and widths.
    These have to be the same length.
    Each pair corresponds to the width at that altitude.
    Gives more control over the geometry.

In [ ]:
# Create a complex bed.
bed = GlacierBed(altitudes=(3800, 3300, 3000, 2100),
                 widths=(1200, 800, 800, 600), slope=0.2)
# Create a simple bed
# bed = GlaGlacierBed(top=3800, bottom=2100, width=300)

In [ ]:
# Representation
bed

In [ ]:
# Plotting method.
bed.plot()

## Glacier
The `Glacier` class gives us an object with methods and attributes fitting for a  glacier.
For now it is pretty simple, but will be extended.
We begin with the definition of our glacier.
It expects a bed - of the type `GlacierBed`.

In [ ]:
# Initialise the glacier
my_glacier = Glacier(bed)

If we simply print the glacier we get some basic statistics about it

In [ ]:
# Prints what is currently known about the glacier.
my_glacier

Since we haven't actually grown the glacier yet, the glacier has no volume etc. Note also that we don't have an ELA yet. We'll add this later.

The glacier also has a plotting method

In [ ]:
# A simple plotting method of the current glacier state.
my_glacier.plot()

In [ ]:
# This will raise an error
my_glacier.plot_history()

Before we can grow the glacier we have to define the mass balance of the glacier. This is accomplished by setting the ELA and  mb_gradient attributes. When both are set, internally the mass balance model will be initiated. These can be set in any order.

In [ ]:
# Set the ela to 3000 m.
my_glacier.ELA = 3500

In [ ]:
# This will not return anything
my_glacier.mb_model

In [ ]:
# Set the mass balance gradient to 7 mm/m
my_glacier.mb_gradient = 7

In [ ]:
# Now this will return the oggm mass balance description.
my_glacier.mb_model

With the mass balance, we can let the glacier grow. There are two methods for this. `Glacier.grow_to_year(year=)`  grows the glacier until the specified year.

In [ ]:
# Grow the glacier until year 60
my_glacier.progress_to_year(60)

In [ ]:
my_glacier.age

In [ ]:
# Outputs have now been updated.
my_glacier

In [ ]:
# Plotting the glacier now has more information. Glacier surface and ELA.
my_glacier.plot()

#### Copy a glacier
We can also initialise a new glacier based on an already existing glacier. Simply provide a glacier under the `copy` keyword to the glacier class:

In [ ]:
glacier2 = Glacier(copy=my_glacier)

This glacier can now be progressed/changed independently from the original glacier.

In [ ]:
# Change the sliding parameter
glacier2.basal_sliding = 5.7e-20
glacier2.progress_to_year(150)
glacier2.plot()

In [ ]:
# Progress the original glacier to year 150
my_glacier.progress_to_year(150)
my_glacier.plot()

## Take a look at the history of the glacier
For this the glacier has the history attribute.
This is a dataset that contains some useful diagnostics about the glacier.

In [ ]:
my_glacier.history

We can plot the history of some attributes of the glacier quickly

In [ ]:
my_glacier.plot_history()

Grow it for a little longer

In [ ]:
my_glacier.progress_to_year(210)
my_glacier.plot()

In [ ]:
my_glacier.plot_history()

The glacier also has a method that progress it to equilibrium `Glacer.progress_to_equilibrium()`

In [ ]:
# Grow the glacier to equilbrium
my_glacier.progress_to_equilibrium()

In [ ]:
# Plot the glacier again.
my_glacier.plot()

There is a method for plotting the mass balance.

In [ ]:
my_glacier.plot_mass_balance()

If we change any of the mass balance parameters, it will internally update the mass balance of the glacier. Growing the glacier will start in the previous state but with the new mass balance. Hence, the glacier will either shrink or grow depending on what is changed. As of now, the glacier is not saving the previous states so one can not compare the glacier before and after the change. One could imagine that glacier to have a mechanism that saves states if the mass balance is changed or something similar. But it also nice to keep the glacier class as light as possible, which is why we have the glacier collection. 

Other methods/tweaks that could be interesting for the glacier class:
- ~~Grow and save in order to plot the evolution~~
- ~~Surging, however this might be it's own subclass.~~ See further down.
- Response time

Let's raise the ELA as a test

In [ ]:
# Raise the ELA
my_glacier.ELA = 3700

In [ ]:
# Grow to the new equilbrium
my_glacier.progress_to_equilibrium()

In [ ]:
# Plot it
my_glacier.plot()

In [ ]:
my_glacier.plot_history()

The glacier is now quite small...

# The the glacier collection
The `GlacierCollection()` class is made to make it simple to work with multiple glaciers. As of now, we have to define the glaciers separately and put them in the collection. But it is not hard to imagine that it could also have a method to populate it with a number of glaciers.

In [ ]:
# This initiates an empty collection.
collection = GlacierCollection()

In [ ]:
collection

In [ ]:
# This conveniently inherits the print of the glacier object. Maybe this
# will get annoying with many glaciers.
# As of now, it is empty.
collection.glaciers

Let's create another two glaciers to add to the collection

In [ ]:
# lets create two other glaciers.
bed = GlacierBed(top=3400, bottom=1400, width=300)
# They have the same bed.
glacier1 = Glacier(bed)
glacier1.ELA = 3000
# But different mb gradients
glacier1.mb_gradient = 7
# lets create two other glaciers.
# Here we copy the first one, but one could just define new ones a done
# above.
glacier2 = Glacier(copy=glacier1)
glacier3 = Glacier(copy=glacier1)

We add glaciers to the collection with the `.add()` method:

In [ ]:
# Add the first glacier
collection.add([glacier1, glacier2, glacier3])
# Change some ice flow stuff
# No problem to change the glaciers within the collection
# "outside" of the collection.
glacier2.creep = glacier2.creep * 10
glacier3.creep = glacier3.creep / 10

In [ ]:
# Now this has some content
collection.glaciers

In [ ]:
# One can also add the glaciers to the collection separately.
# collection.add(glacier1)
# collection.add(glacier2)

In [ ]:
# The collection now has some glaciers in it.
collection

As a glacier, the glacierCollection also has a `.plot()` method.

In [ ]:
collection.plot()

But our glaciers haven't grown anything yet so this will be empty. The collection has the same methods for growing as the individual glaciers

In [ ]:
# Grow the glacier in the collection to year ...
collection.progress_to_year(300)

In [ ]:
# If we plot it again
collection.plot()

And we can grow the glaciers until equilibrium

In [ ]:
collection.progress_to_equilibrium()

In [ ]:
collection.plot()

In [ ]:
collection.plot_history()

In [ ]:
collection.glaciers[1].ELA = 3200

In [ ]:
#  Glaciers in collection will now have different ages.
collection.progress_to_equilibrium()

In [ ]:
collection.plot_history()

As with the `Glacier` class there are probably more things we can add to the `GlacierCollection`, e.g.
- Method to fill it with n glaciers.
- Plot mass balances?
- ???

## Surging Glacier
 The surging glacier is provided by another class. It behaves mostly like the glacier, but with some tweaks

In [ ]:
# Create a complex bed.
bed = GlacierBed(altitudes=(3800, 3300, 3000, 2100),
                 widths=(1200, 800, 800, 600), slope=0.2)
# We're using the bed from earlier.
surging_glacier = SurgingGlacier(bed)

In [ ]:
# Same as the Glacier 
surging_glacier.ELA = 3600
surging_glacier.mb_gradient = 7
surging_glacier.basal_sliding = 5.7e-20
# Some of the new attributes of the surging glacier
surging_glacier.normal_years = 100
surging_glacier.surging_years = 10
surging_glacier.basal_sliding_surge = 5.7e-20 * 10

In [ ]:
surging_glacier.progress_to_year(200)
surging_glacier.plot()

In [ ]:
# Plot the history
surging_glacier.plot_history()

In [ ]:
# We can easlity to continue the progress of the glacier, this does not
# re-run the glacier, but continues from the current state.
surging_glacier.progress_to_year(500)
surging_glacier.plot_history()

In [ ]:
# Surging glaciers do not have the progress_to_equilibrium method.
surging_glacier.progress_to_equilibrium()